In [2]:
import requests
from bs4 import BeautifulSoup
import re
import urllib
import pandas as pd
import numpy as np
import os

All data is imported from http://isophonics.net/content/reference-annotations-beatles. RDF file can be played with mp3 using https://www.sonicvisualiser.org/.

In [3]:
req = requests.get('http://isophonics.net/content/reference-annotations-beatles')
soup = BeautifulSoup(req.text)

In [4]:
#Parse url in order to find links to download csv

pattern = '[0-9\.\'\(\)A-Za-z- ]+'
songs = []
structural_segmentation = []
key_changes = []
chords = []
beats = []
for i in range(len(soup.find_all('ol'))):
    for k in range(len(soup.find_all('ol')[i].find_all('li'))):
        chunk = soup.find_all('ol')[i].find_all('li')[k]
        title = chunk.text
        song = re.findall(pattern, title)[0]
        if (song == 'Structural segmentation'):
            structural_segmentation.append(chunk.find('a').get('href'))
        elif (song == 'Key changes'):
            key_changes.append(chunk.find('a').get('href'))
        elif (song == 'Chords'):
            chords.append(chunk.find('a').get('href'))
        elif (song == 'Beats'):
            beats.append(chunk.find('a').get('href'))
        elif (song != 'All of the above'):
                songs.append(song)
                if song == 'Revolution 9':
                    key_changes.append('none')
                    beats.append('none')
                    
                              

In [5]:
song_info = pd.DataFrame({'songs':songs, 'structural_segmentation': structural_segmentation, 'key_changes': key_changes, 'chords': chords, 'beats':beats})
def get_info(song, data):
    """
    Inputs song name and type of data needed, returns link to input into following functions.
    """
    return song_info.loc[song_info["songs"] == song].get(data).item()


In [6]:
def parse_structural_segmentation(inp):
    """
    Inputs url to txt file for structural segmentation and outputs dataframe. 
    """
    inp = inp.replace(' ', '%20')
    url = 'http://isophonics.net' + inp
    file = urllib.request.urlopen(url)
    data = pd.read_csv(file, sep="\t", header=None, names=list(range(5)))
    return data
    data = data.drop(2, axis=1)
    data.columns = ['begin', 'end', 'structure', 'note']
    return data

In [7]:
def parse_key_changes(inp):
    """
    Inputs url to txt file for key changes and outputs dataframe. 
    """
    inp = inp.replace(' ', '%20')
    url = 'http://isophonics.net' + inp
    file = urllib.request.urlopen(url)
    data = pd.read_csv(file, sep="\t", header=None, names=list(range(2)))
    data.columns = ['begin', 'end', 'key or silence', 'key']
    return data

In [8]:
def parse_beats(inp):
    """
    Inputs url to txt file for beats and outputs dataframe. 
    """
    inp = inp.replace(' ', '%20')
    url = 'http://isophonics.net' + inp
    file = urllib.request.urlopen(url)
    data = pd.read_csv(file, sep="\t", header=None, names=list(range(2)))
    data.columns = ['time', 'beats']
    return data
    if data["beats"][0] != "New Point":
        if np.isnan(int(data["beats"][0])):
            spl = data["time"].apply(lambda x: x.split())
            data["time"] = spl.apply(lambda x: x[0])
            data["beats"] = spl.apply(lambda x: x[1])
    return data

In [9]:
def parse_chords(inp):
    """
    Inputs url to txt file for chords and outputs dataframe. 
    """
    inp = inp.replace(' ', '%20')
    url = 'http://isophonics.net' + inp
    file = urllib.request.urlopen(url)
    data = pd.read_csv(file, sep=" ", header=None, names=list(range(3)))
    data.columns = ['begin', 'end', 'chord']
    return data

In [18]:
#requires directory named data, subdirectory metadata, and directories structural_segmentation, key_changes, beats, chords
# path = os.path.join("data", 'metadata/')
# for song in song_info["songs"]:
#         ss_link = get_info(song, "structural_segmentation")
#         ss_df = parse_structural_segmentation(ss_link)
#         ss_df.to_csv(path+'structural_segmentation/' + song + '_ss.csv', header=True, index=False)
#         if get_info(song, "key_changes") != 'none':
#             kc_link = get_info(song, "key_changes")
#             kc_df = parse_key_changes(kc_link)
#             kc_df.to_csv(path+'key_changes/' + song + '_kc.csv', header=True, index=False)
#         if get_info(song, "beats") != 'none':
#             beats_link = get_info(song, "beats")
#             beats_df = parse_beats(beats_link)
#             beats_df.to_csv(path+'beats/' + song + '_beats.csv', header=True, index=False)
#         chords_link = get_info(song, "chords")
#         chords_df = parse_chords(chords_link)
#         chords_df.to_csv(path+'chords/' + song + '_chords.csv', header=True, index=False)

        